In [1]:
# this notebook will give you the appotunity to work with dynamic data

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import trimboth
import ast
import time
import datetime
import geopy
from tqdm import tqdm
import geopy.distance
from tqdm._tqdm_notebook import tqdm_notebook
tqdm.pandas(desc="Bar")

In [3]:
import plotly.offline as pyo
import chart_studio.plotly as chspy
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.express as px

init_notebook_mode(connected=True)

In [4]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import csv

In [5]:
flats = pd.read_csv('final.csv', index_col=0)

In [45]:
spb = flats[(flats.trade_type == 'sale')&\
            ((flats.city == 'Санкт-Петербург')| (flats.city == 'Ленинградская область'))]

In [46]:
spb['date'] = spb.date_of_adding_to_db

In [47]:
#made list of unique dates
date_list = spb.date.unique()

In [48]:
# for every cian_id made column which tells when ads became passive
spb = pd.merge(spb, spb[spb.active==False].groupby('cian_id')['date'].min().reset_index().rename(columns={'date':'min_passive_date'}),
               on='cian_id', how='left')

In [44]:
#for a certain day found max active date
# max_active_ads = spb[((date_list[53]<spb.min_passive_date)| pd.isnull(spb.min_passive_date)) & \
#    (spb.active==True) & (spb.date<=date_list[53])].groupby('cian_id')['date'].max().reset_index()

In [30]:
#made merge 
# pd.merge(spb, max_active_ads, on=['cian_id', 'date']).head()

In [49]:
date_dict = {}

In [50]:
for date in tqdm(date_list):
    max_active_ads = spb[((date<spb.min_passive_date)| pd.isnull(spb.min_passive_date)) & \
   (spb.active==True) & (spb.date<=date)].groupby('cian_id')['date'].max().reset_index()
    date_dict[pd.to_datetime(date)] = pd.merge(spb, max_active_ads, on=['cian_id', 'date'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:17<00:00,  3.14it/s]


In [51]:
for i in date_dict:
    print(i, date_dict[i].cian_id.count())

2020-02-27 00:00:00 22
2020-02-28 00:00:00 112
2020-02-29 00:00:00 212
2020-03-01 00:00:00 410
2020-03-02 00:00:00 500
2020-03-03 00:00:00 547
2020-03-04 00:00:00 579
2020-03-05 00:00:00 698
2020-03-06 00:00:00 719
2020-03-07 00:00:00 760
2020-03-08 00:00:00 812
2020-03-09 00:00:00 874
2020-03-10 00:00:00 931
2020-03-11 00:00:00 995
2020-03-12 00:00:00 997
2020-03-13 00:00:00 1017
2020-03-14 00:00:00 1017
2020-03-15 00:00:00 1022
2020-03-18 00:00:00 1079
2020-03-19 00:00:00 1128
2020-03-20 00:00:00 1402
2020-03-21 00:00:00 1402
2020-03-22 00:00:00 1510
2020-03-24 00:00:00 1544
2020-03-26 00:00:00 1545
2020-03-27 00:00:00 1648
2020-03-28 00:00:00 1694
2020-03-29 00:00:00 2016
2020-03-30 00:00:00 2163
2020-03-31 00:00:00 2227
2020-04-01 00:00:00 2231
2020-04-02 00:00:00 2231
2020-04-03 00:00:00 2231
2020-04-04 00:00:00 2231
2020-04-05 00:00:00 2232
2020-04-07 00:00:00 2244
2020-04-08 00:00:00 2326
2020-04-09 00:00:00 2288
2020-04-10 00:00:00 2229
2020-04-12 00:00:00 2302
2020-04-13 00:00

In [56]:
one_flats = []
two_flats = []
x = []

In [57]:
for i in date_dict: 
    x.append(i)
    temp_data = date_dict[i] 
    one_flats.append(temp_data[temp_data.Number_of_rooms==1]['price_per_sq_meter'].mean())

In [58]:
for i in date_dict: 
#     x.append(i)
    temp_data = date_dict[i] 
    two_flats.append(temp_data[temp_data.Number_of_rooms==2]['price_per_sq_meter'].mean())

In [37]:
type(x[0])

pandas._libs.tslibs.timestamps.Timestamp

In [59]:
px.line(x = x, y=two_flats)